In [635]:
import numpy as np

In [636]:
inp_c, inp_h, inp_w = 2, 4, 4 # input channels, input height, input weight
ker_c, ker_h, ker_w = 2, 2, 2 # kernel channels, kernel height, kernel weight
s = 1 # stride
num_ker = 3 # number of kernels
dilation = 1 # dilation factor
print('input channels: {}, input height: {}, input weight: {}'.format(inp_c, inp_h, inp_w))
print('kernel channels: {}, kernel height: {}, kernel weight: {}'.format(ker_c, ker_h, ker_w))

input channels: 2, input height: 4, input weight: 4
kernel channels: 2, kernel height: 2, kernel weight: 2


In [637]:
input_img = np.random.rand(inp_c, inp_h, inp_w) # define a random image based on the input parameters
input_img = np.ones_like(input_img) # define an image of all ones based on the input parameters
print('input image')
print(input_img)

input image
[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]


In [638]:
# method to dilate a kernel
def dilate_kernel(dilation, kernels):
    dil_ker_h = dilation * (ker_h - 1) + 1
    dil_ker_w = dilation * (ker_w - 1) + 1
    dil_kernels = []
    for kernel in kernels:
        dil_kernel = []
        for channel in kernel:
            dil_channel = np.zeros((dil_ker_h, dil_ker_w))
            for row in range(len(channel)):
                for col in range(len(channel[0])):
                    dil_channel[dilation*row][dilation*col] = channel[row][col]
            dil_kernel.append(dil_channel.tolist())
        dil_kernels.append(dil_kernel)
    return dil_kernels, dil_ker_h, dil_ker_w

In [647]:
# define random kernels based on the input kernel parameters
kernels = []
print('{} kernels'.format(num_ker))
for k in range(num_ker):
    kernel = np.random.rand(ker_c, ker_h, ker_w) # define a random kernel based on the kernel parameters
    kernel = k * np.ones_like(kernel)
    kernels.append(kernel)
    print('kernel {}'.format(k))
    print(kernel)
print('kernel channels: {}, kernel height: {}, kernel weight: {}'.format(ker_c, ker_h, ker_w))

3 kernels
kernel 0
[[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
kernel 1
[[[1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]]]
kernel 2
[[[2. 2.]
  [2. 2.]]

 [[2. 2.]
  [2. 2.]]]
kernel channels: 2, kernel height: 2, kernel weight: 2


In [648]:
# dilate kernels
kernels, ker_h, ker_w = dilate_kernel(dilation, kernels)
for k in range(num_ker):
    print('dilated kernel {}'.format(k))
    print(kernels[k])
print('dilated kernel channels: {}, dilated kernel height: {}, dilated kernel weight: {}'.format(ker_c, ker_h, ker_w))

dilated kernel 0
[[[0.0, 0.0], [0.0, 0.0]], [[0.0, 0.0], [0.0, 0.0]]]
dilated kernel 1
[[[1.0, 1.0], [1.0, 1.0]], [[1.0, 1.0], [1.0, 1.0]]]
dilated kernel 2
[[[2.0, 2.0], [2.0, 2.0]], [[2.0, 2.0], [2.0, 2.0]]]
dilated kernel channels: 2, dilated kernel height: 2, dilated kernel weight: 2


In [640]:
# method to compute output volume from the input and kernel parameters
def compute_out_vol(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker):
    out_c = int(num_ker)
    out_h = int((inp_h - ker_h)/s) + 1
    out_w = int((inp_w - ker_h)/s) + 1
    return out_c, out_h, out_w

In [649]:
out_c, out_h, out_w = compute_out_vol(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker)
print('ouput channels: {}, output height: {}, output weight: {}'.format(out_c, out_h, out_w))

ouput channels: 3, output height: 3, output weight: 3


In [650]:
# method to convolve kernel over the input slices
def convolve(c, h, w, ker_c, ker_h, ker_w, s, ker_num):
    print('kernel indices, image indices')
    print('[c, h, w]', '[c, h, w]')
    convol_sum = 0
    for c_ker in range(ker_c):
        for h_ker in range(ker_h):
            for w_ker in range(ker_w):
                print([c_ker, h_ker, w_ker], [c_ker, h_ker + s*h, w_ker + s*w])
#                 convol_sum += kernels[ker_num][c_ker, h_ker, w_ker] * input_img[c_ker, h_ker + s*h, w_ker + s*w]
                convol_sum += kernels[ker_num][c_ker][h_ker][w_ker] * input_img[c_ker][h_ker + s*h][w_ker + s*w]
    print('\n')
    return convol_sum

In [651]:
# method to create output from the input and kernel parameters 
def create_output(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker):
    out_c, out_h, out_w = compute_out_vol(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker)
    output = np.zeros([out_c, out_h, out_w])
    # parse through every element of the output and compute the convolution value for that element
    for k in range(num_ker):
        for h in range(out_h):
            for w in range(out_w):
                for c in range(inp_c):
                    output[k, h, w] += convolve(c, h, w, ker_c, ker_h, ker_w, s, k)
    return output

In [652]:
output = create_output(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker)
print('output')
output_shape = output.shape
print('ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2]))
assert((out_c, out_h, out_w) == output_shape)
print(output)

kernel indices, image indices
[c, h, w] [c, h, w]
[0, 0, 0] [0, 0, 0]
[0, 0, 1] [0, 0, 1]
[0, 1, 0] [0, 1, 0]
[0, 1, 1] [0, 1, 1]
[1, 0, 0] [1, 0, 0]
[1, 0, 1] [1, 0, 1]
[1, 1, 0] [1, 1, 0]
[1, 1, 1] [1, 1, 1]


kernel indices, image indices
[c, h, w] [c, h, w]
[0, 0, 0] [0, 0, 0]
[0, 0, 1] [0, 0, 1]
[0, 1, 0] [0, 1, 0]
[0, 1, 1] [0, 1, 1]
[1, 0, 0] [1, 0, 0]
[1, 0, 1] [1, 0, 1]
[1, 1, 0] [1, 1, 0]
[1, 1, 1] [1, 1, 1]


kernel indices, image indices
[c, h, w] [c, h, w]
[0, 0, 0] [0, 0, 1]
[0, 0, 1] [0, 0, 2]
[0, 1, 0] [0, 1, 1]
[0, 1, 1] [0, 1, 2]
[1, 0, 0] [1, 0, 1]
[1, 0, 1] [1, 0, 2]
[1, 1, 0] [1, 1, 1]
[1, 1, 1] [1, 1, 2]


kernel indices, image indices
[c, h, w] [c, h, w]
[0, 0, 0] [0, 0, 1]
[0, 0, 1] [0, 0, 2]
[0, 1, 0] [0, 1, 1]
[0, 1, 1] [0, 1, 2]
[1, 0, 0] [1, 0, 1]
[1, 0, 1] [1, 0, 2]
[1, 1, 0] [1, 1, 1]
[1, 1, 1] [1, 1, 2]


kernel indices, image indices
[c, h, w] [c, h, w]
[0, 0, 0] [0, 0, 2]
[0, 0, 1] [0, 0, 3]
[0, 1, 0] [0, 1, 2]
[0, 1, 1] [0, 1, 3]
[1, 0, 0] [1, 0, 2]
[1